In [1]:
import platform
import sys
import platform
import psutil
import GPUtil
import platform
import psutil
import subprocess
import re
# import torch

import psutil
import platform

import platform
import socket
import re
import uuid
import json
import psutil
import logging
import pandas as pd

In [2]:
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

System: Windows
Node Name: DELL-G15
Release: 10
Version: 10.0.22621
Machine: AMD64
Processor: Intel64 Family 6 Model 154 Stepping 3, GenuineIntel


# Greneral

In [3]:
import platform
import socket
import re
import uuid
import json
import psutil
import logging
import pandas as pd

def get_graphics_memory():
    system = platform.system()
    
    if system == 'Windows':
        import wmi
        
        try:
            wmi_obj = wmi.WMI()
            gpu_info = wmi_obj.Win32_VideoController()[0]
            total_memory = int(gpu_info.AdapterRAM) / 1024 / 1024  # Convert to MB
            memory_size = total_memory if total_memory < 1024 else total_memory / 1024  # Convert to GB if >= 1024 MB
            return total_memory, memory_size
        except Exception as e:
            print(f"Error retrieving graphics memory on Windows: {str(e)}")
    
    elif system == 'Linux':
        try:
            with open('/var/log/Xorg.0.log', 'r') as file:
                content = file.read()
                
            memory_lines = [line for line in content.split('\n') if 'Memory:' in line]
            if memory_lines:
                memory_line = memory_lines[-1]
                total_memory = int(memory_line.split('Memory:')[1].split('kB')[0].strip()) / 1024  # Convert to MB
                memory_size = total_memory if total_memory < 1024 else total_memory / 1024  # Convert to GB if >= 1024 MB
                return total_memory, memory_size
            else:
                print("Graphics memory information not found in Xorg log file.")
        except Exception as e:
            print(f"Error retrieving graphics memory on Linux: {str(e)}")
    
    elif system == 'Darwin':
        try:
            command = "system_profiler SPDisplaysDataType | grep 'VRAM (Total)'"
            output = psutil.Popen(command, shell=True, stdout=psutil.PIPE).stdout.read().decode()
            total_memory = int(output.split(':')[1].strip()) / 1024  # Convert to MB
            memory_size = total_memory if total_memory < 1024 else total_memory / 1024  # Convert to GB if >= 1024 MB
            return total_memory, memory_size
        except Exception as e:
            print(f"Error retrieving graphics memory on macOS: {str(e)}")
    
    else:
        print("Unsupported operating system.")
    
    return None, None

# Example usage:
total_memory, memory_size = get_graphics_memory()
if total_memory and memory_size:
    print(f"Total Graphics Memory: {total_memory} MB")
    print(f"Graphics Size: {memory_size} GB" if memory_size >= 1 else f"Graphics Size: {memory_size} MB")


def getSystemInfo():
    try:
        info = {}            
        info['platform'] = platform.system()
        info['platform-release'] = platform.release()
        info['platform-version'] = platform.version()
        info['architecture'] = platform.machine()
        info['hostname'] = socket.gethostname()
        info['processor'] = platform.processor()
        info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3))) + " GB"        
        total_memory, memory_size = get_graphics_memory()
        if total_memory and memory_size:
            info['Graphic'] = f'{total_memory} MB'
            info['Graphic_'] = f'{memory_size} GB'
        df = pd.DataFrame.from_dict(info, orient='index', columns=['Value']).rename_axis('Name').reset_index()
        filename = f'system_info_Min{platform.system()}.csv'
        df.to_csv(filename, index=False)
        return df
    except Exception as e:
        logging.exception(e)
        
getSystemInfo()

# system_info = getSystemInfo()pd.DataFrame.from_dict(system_info, orient='index', columns=['Value']).rename_axis('Name').reset_index()

# Convert system information to DataFrame
# df = pd.DataFrame.from_dict(system_info, orient='index', columns=['Value']).rename_axis('Name').reset_index()

# # Save system information to a CSV file
# filename = 'system_info.csv'
# df.to_csv(filename, index=False)
# df


Total Graphics Memory: 1024.0 MB
Graphics Size: 1.0 GB


,Name,Value
0,platform,Windows
1,platform-release,10
2,platform-version,10.0.22621
3,architecture,AMD64
4,hostname,DELL-G15
5,processor,"Intel64 Family 6 Model 154 Stepping 3, Genuine..."
6,ram,32 GB
7,Graphic,1024.0 MB
8,Graphic_,1.0 GB


In [4]:
import psutil
import platform
import cpuinfo
import socket
import uuid
import re
import pandas as pd

# GPU information
import GPUtil
from tabulate import tabulate


def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
        
def get_Unit(mem): 
    if mem >= 1024:
        graphics_memory = round(mem / 1024, 2)  # Convert to GB
    else:
        graphics_memory = round(mem, 2)  # Keep in MB
    unit = "GB" if graphics_memory >= 1 else "MB"
    
    return f"{graphics_memory} {unit}"

def get_gpu_info_linux():
    try:
        output = subprocess.check_output(['nvidia-smi', '--query-gpu=index,name,memory.total,memory.used,memory.free,temperature.gpu', '--format=csv,noheader,nounits'])
        output = output.decode('utf-8').strip()
        gpu_info = output.split('\n')
        gpu_info = [info.split(',') for info in gpu_info]
        return gpu_info
    except (subprocess.CalledProcessError, FileNotFoundError):
        return []


def System_information():
    info = []

    syss = platform.architecture(executable=sys.executable, bits='', linkage='')
    uname = platform.uname()
    info.append(("System", uname.system))
    info.append(("Node Name", uname.node))
    info.append(("Release", uname.release))
    info.append(("Version", uname.version))
    info.append(("Machine", uname.machine))
    info.append(("Arch", syss[0]))
    info.append(("Processor", uname.processor))
    info.append(("Processor", cpuinfo.get_cpu_info()['brand_raw']))

    cpufreq = psutil.cpu_freq()
    info.append(("Max Frequency", f"{cpufreq.max:.2f}Mhz"))
    info.append(("Min Frequency", f"{cpufreq.min:.2f}Mhz"))
    info.append(("Current Frequency", f"{cpufreq.current:.2f}Mhz"))

    svmem = psutil.virtual_memory()
    info.append(("Total Memory", get_size(svmem.total)))
    # info.append(("Available Memory", get_size(svmem.available)))
    # info.append(("Used Memory", get_size(svmem.used)))
    # info.append(("Memory Percentage", f"{svmem.percent}%"))

    swap = psutil.swap_memory()
    info.append(("Total Swap Memory", get_size(swap.total)))
#     info.append(("Free Swap Memory", get_size(swap.free)))
#     info.append(("Used Swap Memory", get_size(swap.used)))
#     info.append(("Swap Memory Percentage", f"{swap.percent}%"))
    
    
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        for gpu in gpus:
            gpu_id = gpu.id
            gpu_name = gpu.name
            # gpu_load = f"{gpu.load*100}%"
            # gpu_free_memory = get_Unit(gpu.memoryFree)
            # gpu_used_memory = get_Unit(gpu.memoryUsed)
            gpu_total_memory = get_Unit(gpu.memoryTotal)
            # gpu_temperature = f"{gpu.temperature} °C"
            # gpu_uuid = gpu.uuid

            info.append(("GPU ID", gpu_id))
            info.append(("GPU Name", gpu_name))
            # info.append(("GPU Load", gpu_load))
            # info.append(("GPU Free Memory", gpu_free_memory))
            # info.append(("GPU Used Memory", gpu_used_memory))
            info.append(("GPU Total Memory", gpu_total_memory))
            # info.append(("GPU Temperature", gpu_temperature))
            # info.append(("GPU UUID", gpu_uuid))

    except ImportError:
        info.append(("GPU Information", "N/A"))
        
        
    
    try:
        gpu_info = get_gpu_info_linux()
        for i, gpuL in enumerate(gpu_info):
            gpu_id = gpuL[0]
            gpu_name = gpuL[1]
            gpu_total_memory = get_Unit(int(gpuL[2]))
            # gpu_used_memory = get_Unit(int(gpuL[3]))
            # gpu_free_memory = get_Unit(int(gpuL[4]))
            # gpu_temperature = f"{gpuL[5]} °C"

            info.append(("GPU ID", gpu_id))
            info.append(("GPU Name", gpu_name))
            # info.append(("GPU Load", gpu_load))
            # info.append(("GPU Free Memory", gpu_free_memory))
            # info.append(("GPU Used Memory", gpu_used_memory))
            info.append(("GPU Total Memory", gpu_total_memory))
            # info.append(("GPU Temperature", gpu_temperature))

    except ImportError:
        pass
    
    
    info.append(("Python Version", platform.python_version()))



    df = pd.DataFrame(info, columns=["Name", "Value"])
    df.to_csv(f"system_info_{uname.system}.csv", index=False)

    return df


System_information()


,Name,Value
0,System,Windows
1,Node Name,DELL-G15
2,Release,10
3,Version,10.0.22621
4,Machine,AMD64
5,Arch,64bit
6,Processor,"Intel64 Family 6 Model 154 Stepping 3, Genuine..."
7,Processor,12th Gen Intel(R) Core(TM) i7-12700H
8,Max Frequency,2300.00Mhz
9,Min Frequency,0.00Mhz


In [5]:
import subprocess
import re

def get_graphic_memory_linux():
    try:
        lspci_output = subprocess.check_output(["lspci", "-v"], universal_newlines=True)
        gpu_matches = re.findall(r"VGA compatible controller:\s(.+)", lspci_output)
        if gpu_matches:
            gpu_name = gpu_matches[0]

            meminfo_output = subprocess.check_output(["grep", "-i", "vram", "/var/log/Xorg.0.log"], universal_newlines=True)
            meminfo_match = re.search(r"System memory:\s+(\d+)MB", meminfo_output)
            if meminfo_match:
                gpu_memory = int(meminfo_match.group(1)) / 1024  # Convert to GB
                gpu_memory = round(gpu_memory, 2)
                return gpu_name, gpu_memory
    except Exception as e:
        pass
    
    return None, None

# Usage
gpu_name, gpu_memory = get_graphic_memory_linux()
if gpu_name and gpu_memory:
    print("GPU Name:", gpu_name)
    print("GPU Memory:", gpu_memory, "GB" if gpu_memory >= 1 else "MB")
else:
    print("GPU information not found.")


GPU information not found.


In [6]:
import platform
import psutil
import re
import subprocess
import pandas as pd
# import wmi

def get_system_properties():
    properties = {}

    # Get OS information
    os_name = platform.system()
    os_version = platform.release()
    properties["OS Name"] = os_name
    properties["OS Version"] = os_version

    if os_name == "Windows":
        import wmi
        c = wmi.WMI()

        # Get Windows version name
        os_version_info = platform.win32_ver()
        os_version = os_version_info[0] if os_version_info[0] else os_version
        properties["OS Version"] = os_version

        # Get Windows specific GPU information
        try:          
            gpu_name = ""
            gpu_memory = ""
            gpu_info = c.Win32_VideoController()[0]
            gpu_name = gpu_info.Name          
            
            
            gpu_memory = gpu_info.AdapterRAM / (1024 ** 3)  # GPU memory size in GB (gigabytes)
            if gpu_memory >= 1:
                gpu_memory = round(gpu_memory, 2)
            else:
                gpu_memory = round(gpu_memory * 1024, 2)  # Convert to MB (megabytes)
            properties["GPU Name"] = gpu_name
            properties["GPU Memory"] = f"{gpu_memory} GB" if gpu_memory >= 1 else f"{gpu_memory} MB"
        except Exception:
            pass


    elif os_name == "Linux":
        # Get Linux version information
        os_version_info = platform.uname()
        os_version = os_version_info[2] if os_version_info[2] else os_version
        properties["OS Version"] = os_version

        # Get Linux specific GPU information
        try:            
            gpu_name = ""
            gpu_memory = ""
            lspci_output = subprocess.check_output(["lspci", "-v"], universal_newlines=True)
            gpu_match = re.search(r"VGA compatible controller:\s(.+)", lspci_output)
            if gpu_match:
                gpu_name = gpu_match.group(1)

                meminfo_output = subprocess.check_output(["grep", "-i", "vram", "/var/log/Xorg.0.log"],
                                                         universal_newlines=True)
                meminfo_match = re.search(r"System memory:\s+(\d+)MB", meminfo_output)
                if meminfo_match:
                    gpu_memory = int(meminfo_match.group(1)) / 1024  # Convert to GB
                    gpu_memory = round(gpu_memory, 2)
            properties["GPU Name"] = gpu_name
            properties["GPU Memory"] = f"{gpu_memory} GB" if gpu_memory >= 1 else f"{gpu_memory} MB"
        except Exception:
            pass
        

    # Get processor information
    processor_name = platform.processor()
    num_cpu_cores = psutil.cpu_count(logical=False)  # Number of physical CPU cores
    properties["Processor Name"] = processor_name
    properties["Number of CPU Cores"] = num_cpu_cores

    # Get memory information
    ram_size = psutil.virtual_memory().total / (1024 ** 3)  # RAM size in GB (gigabytes)
    if ram_size >= 1:
        ram_size = round(ram_size, 2)
    else:
        ram_size = round(ram_size * 1024, 2)  # Convert to MB (megabytes)
    properties["RAM Size"] = f"{ram_size} GB" if ram_size >= 1 else f"{ram_size} MB"

    # Get Python version
    properties["Python Version"] = platform.python_version()
    
    df = pd.DataFrame(properties.items(), columns=['Property', 'Value'])
    return df
    
get_system_properties()

,Property,Value
0,OS Name,Windows
1,OS Version,10
2,GPU Name,Intel(R) Iris(R) Xe Graphics
3,GPU Memory,1.0 GB
4,Processor Name,"Intel64 Family 6 Model 154 Stepping 3, Genuine..."
5,Number of CPU Cores,14
6,RAM Size,31.69 GB
7,Python Version,3.9.13


In [7]:
import pandas as pd
import platform

def get_memory_info():
    system = platform.system()
    
    if system != 'Windows':
        with open('/proc/meminfo') as file:
            for line in file:
                if 'MemFree' in line:
                    free_mem_in_kb = int(line.split()[1])
                    break

        free_mem_in_mb = free_mem_in_kb / 1024

        if free_mem_in_mb >= 1024:
            free_mem = free_mem_in_mb / 1024
            unit = 'GB'
        else:
            free_mem = free_mem_in_mb
            unit = 'MB'

        data = [('Free Memory', f'{free_mem:.2f} {unit}')]
        df = pd.DataFrame(data, columns=['Name', 'Value'])
        return df
    else: 
        pass
memory_info = get_memory_info()
print(memory_info)

None
